# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [12]:
!pip install openai panel python-dotenv


In [14]:
!pip install jupyter_bokeh


  Using cached jupyter_bokeh-4.0.5-py3-none-any.whl.metadata (7.1 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
Using cached jupyter_bokeh-4.0.5-py3-none-any.whl (148 kB)
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl (214 kB)
Using cached widgetsnbextension-4.0.13-py3-none-any.whl (2.3 MB)
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 1.0.0
    Uninstalling jupyterlab-widgets-1.0.0:
      Successfully uninstalled jupyterlab-widgets-1.0.0
  Attempting uninstall: ipywidgets
    Found existing installation

In [15]:
!conda install jupyter_bokeh


Channels:
 - defaults
Platform: win-64
Solving environment: ...working... failed


warning  libmamba Added empty dependency for problem type SOLVER_RULE_UPDATE

LibMambaUnsatisfiableError: Encountered problems while solving:
  - nothing provides package_has_been_revoked needed by spyder-5.5.1-py310haa95532_1

Could not solve for environment specs
The following packages are incompatible
├─ _anaconda_depends is installable with the potential options
│  ├─ _anaconda_depends [2024.02|2024.06|2024.10|2025.02|2025.03] would require
│  │  ├─ spyder with the potential options
│  │  │  ├─ spyder [5.5.1|6.0.1|6.0.3] would require
│  │  │  │  └─ python-lsp-server [>=1.10.0,<1.11.0 |>=1.12.0,<1.13.0 ] with the potential options
│  │  │  │     ├─ python-lsp-server [1.10.0|1.12.0] would require
│  │  │  │     │  └─ yapf >=0.33.0  with the potential options
│  │  │  │     │     ├─ yapf 0.40.2 would require
│  │  │  │     │     │  └─ importlib-metadata >=6.6.0  with the potential options
│  │  │  │     │     │     ├─ importlib-metadata 7.0.1, which can be installed;
│  │  │  │     │

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

In [3]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [4]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, perchance! 'Tis a classic jest among the fowl and the jesters alike.


In [5]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How can I assist you today?


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I can't remember your name as I'm just a computer program. Could you please remind me?


In [7]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [8]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard.show()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11936\781706477.py:2: UserWarning: Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.
  pn.extension()


Launching server at http://localhost:51577


ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'MessageSent', 'msg_type': 'bokeh_event', 'msg_data': {'type': 'event', 'name': 'button_click', 'values': {'type': 'map', 'entries': [['model', {'id': '69b0e0d2-eb18-4891-9196-25a3a1edcd1a'}]]}}}]} 
 error: NameError("name 'openai' is not defined")
Traceback (most recent call last):
  File "c:\Users\ASUS\anaconda3\Lib\site-packages\bokeh\server\protocol_handler.py", line 94, in handle
    work = await handler(message, connection)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ASUS\anaconda3\Lib\site-packages\bokeh\server\session.py", line 94, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ASUS\anaconda3\Lib\site-packages\bokeh\server\session.py", line 286, in _handle_patch
    message.apply_to_document(self.document, self)
  File "c:\Users\ASUS\anaconda3\Lib\site-package

In [12]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "pepperoni pizza",
    "size": "large"
  },
  "toppings": [
    "extra cheese",
    "mushrooms"
  ],
  "drinks": [
    {
      "type": "coke",
      "size": "medium"
    }
  ],
  "sides": [
    {
      "type": "fries",
      "size": "regular"
    }
  ],
  "total price": 23.45
}


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [17]:
import os
import openai
import panel as pn  # GUI
from dotenv import load_dotenv
from openai import OpenAI  # Import OpenAI Client

# Load API Key from .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # Fetch API key securely

# Ensure API Key exists
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is missing! Make sure your .env file is set up correctly.")

# Initialize OpenAI Client
client = OpenAI(api_key=OPENAI_API_KEY)

# Function to generate responses
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

# Function for handling multi-message conversation
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0): 
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message.content

# Initialize Panel
pn.extension()
panels = []  # Collects chat messages for display

# System message: Chatbot instructions
context = [
    {"role": "system", "content": """
You are OrderBot, an automated service for taking restaurant orders. 
Your job is to guide the customer through their order in a friendly, engaging manner.

**Ordering Process:**
1. Greet the customer and ask how you can help.
2. Ask if they want **Beef** or **Chicken** as the main dish.
3. Ask if they would like to add juice (options: **Orange, Apple, Mango**).
4. Allow customers to modify or add multiple items.
5. Summarize the order and confirm.
6. Ask if it’s for **pickup or delivery** (if delivery, request the address).
7. Confirm the **total price** and payment method (online or on delivery).

**Menu Options:**
- **Main Dishes:** 
  - Beef Meal - **$12.99**
  - Chicken Meal - **$11.99**
- **Juices:** 
  - Orange Juice - **$3.00**
  - Apple Juice - **$3.00**
  - Mango Juice - **$3.50**

**Response Guidelines:**
- Keep responses short, friendly, and conversational.
- Confirm choices clearly to avoid mistakes.
- Summarize orders before finalizing.
- If a customer asks for something unavailable, politely suggest alternatives.
"""}
]

# Function to handle chatbot messages
def collect_messages(_):
    prompt = inp.value_input  # Get user input
    inp.value = ''  # Clear input field
    context.append({'role': 'user', 'content': f"{prompt}"})  # Store user message
    
    response = get_completion_from_messages(context)  # API call
    context.append({'role': 'assistant', 'content': f"{response}"})  # Store chatbot response

    # Display user and assistant messages
    panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)  # Update chat UI

# Create UI components
inp = pn.widgets.TextInput(value="Hi", placeholder="Enter your order here...")
button_conversation = pn.widgets.Button(name="Chat!")

# Bind button click to function
interactive_conversation = pn.bind(collect_messages, button_conversation)

# Create chatbot interface
dashboard = pn.Column(
    inp,  # User input field
    pn.Row(button_conversation),  # Chat button
    pn.panel(interactive_conversation, loading_indicator=True, height=300),  # Chat panel
)

# Run chatbot UI
dashboard.show()


Launching server at http://localhost:51792


AssertionError: 